In [1]:
import pandas as pd

In [2]:
input_dir  = '/Users/backupuser/loglizer/logparser/logs/SCADA/' # The input directory of log file
output_dir = input_dir # The output directory of parsing results
log_file   = 'logs.txt' # The input log file name
log_format = '<TimeDetected>,<TimeReset>,<UnitTitle>,<Content>,<UnitTitleDestination>' # SCADA log format

In [3]:
df_log_parsed_all = pd.read_csv(output_dir+log_file+'_structured.csv')
df_log_struct_all = pd.read_csv(output_dir+log_file+'_templates.csv')

In [4]:
df_log_parsed_all.head()

,LineId,TimeDetected,TimeReset,UnitTitle,Content,UnitTitleDestination,EventId,EventTemplate,ParameterList
0,2,2017-09-01T00:00:48+00:00,NaN,T06,Gen. ext. vent. 2,NaN,c5e58325,Gen<*> <*> vent<*> <*>,"['. ext.', '. 2']"
1,3,2017-09-01T00:01:23+00:00,NaN,T01,Gen. ext. vent. 2,NaN,c5e58325,Gen<*> <*> vent<*> <*>,"['. ext.', '. 2']"
2,4,2017-09-01T00:04:14+00:00,NaN,T06,External power ref.:2000kW,NaN,6f139984,External power ref<*>:<*><*><*><*>kW,"['', '', '.:2000']"
3,5,2017-09-01T00:07:05+00:00,NaN,T09,Gen. ext. vent. 2,NaN,c5e58325,Gen<*> <*> vent<*> <*>,"['. ext.', '. 2']"
4,6,2017-09-01T00:15:50+00:00,NaN,T09,External power ref.:2000kW,NaN,6f139984,External power ref<*>:<*><*><*><*>kW,"['', '', '.:2000']"


In [5]:
df_log_struct_all.head()

,EventId,EventTemplate,Occurrences
0,c5e58325,Gen<*> <*> vent<*> <*>,3107
1,6f139984,External power ref<*>:<*><*><*><*>kW,24108
2,59c74dd9,Nac<*>vent<*><*>,6021
3,cba5200d,Generator <*> in,4714
4,95402390,Running up timeout,147


In [9]:
df_log_parsed = df_log_parsed_all[df_log_parsed_all.UnitTitle == 'T01']
df_log_parsed.index = pd.to_datetime(df_log_parsed.TimeDetected, utc=True)

In [10]:
lst_EventId =  [[i] for i in df_log_parsed.EventId]
df_log_parsed['lst_EventId'] = lst_EventId

df_log_parsed = df_log_parsed.sort_index()
df_1h_logseq = df_log_parsed.lst_EventId.resample('6H').sum()

/Users/backupuser/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
df_1h_logseq[df_1h_logseq == 0]

TimeDetected
2017-11-08 00:00:00+00:00    0
2017-11-26 00:00:00+00:00    0
2017-12-09 00:00:00+00:00    0
Name: lst_EventId, dtype: object

In [21]:
d = {x: i for i, x in enumerate(set(df_log_parsed.EventId))}
lst_new = [d[x] for x in df_log_parsed.EventId]

In [23]:
from loglizer import preprocessing
feature_extractor = preprocessing.FeatureExtractor()
x_train = feature_extractor.fit_transform(df_log_parsed.EventId, term_weighting='tf-idf', 
                                              normalization='zero-mean')

====== Transformed train data summary ======
Train data shape: 15412-by-16



In [26]:
x_train.shape[1]

16

In [28]:
log_feats = pd.DataFrame(data=x_train, index=df_log_parsed.index, columns=['log_ft_{}'.format(i+1) for i in range(x_train.shape[1])])

In [29]:
log_feats.head()

,log_ft_1,log_ft_2,log_ft_3,log_ft_4,log_ft_5,log_ft_6,log_ft_7,log_ft_8,log_ft_9,log_ft_10,log_ft_11,log_ft_12,log_ft_13,log_ft_14,log_ft_15,log_ft_16
TimeDetected,,,,,,,,,,,,,,,,
2017-09-01 00:01:23+00:00,0.277035,1.828561,1.246640,0.237380,0.474226,0.914547,-0.392982,-0.411109,-0.242768,-0.785864,-0.37935,-0.372405,-0.292097,-0.420509,-0.335346,-0.520431
2017-09-01 00:23:41+00:00,0.277035,0.290254,-0.337302,-0.338128,-0.376126,-0.431809,0.325268,0.948551,0.083633,1.309410,-0.37935,-0.372405,-0.292097,-0.420509,-0.335346,-0.520431
2017-09-01 00:23:42+00:00,0.277035,0.290254,-0.337302,-0.338128,-0.376126,-0.431809,0.325268,0.948551,0.083633,1.309410,-0.37935,-0.372405,-0.292097,-0.420509,-0.335346,-0.520431
2017-09-01 00:27:53+00:00,-0.364151,-0.478899,-0.337302,0.237380,0.474226,-0.431809,0.325268,-0.411109,0.083633,-0.785864,0.56654,0.436888,0.143870,-0.420509,-0.335346,-0.520431
2017-09-01 00:28:16+00:00,0.277035,0.290254,-0.337302,-0.338128,-0.376126,-0.431809,0.325268,0.948551,0.083633,1.309410,-0.37935,-0.372405,-0.292097,-0.420509,-0.335346,-0.520431
